# Packages and Functions


In [5]:
import pickle
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np
import pandas as pd

import os
import sys

# Paths
os.chdir("/Users/antoniosquicciarini/ownCloud/PhD Projects/Information_Measurement/")
add_dir = "/Users/antoniosquicciarini/ownCloud/PhD Projects/General_Functions/"
sys.path.insert(1, add_dir)
add_dir = "/Users/antoniosquicciarini/ownCloud/PhD Projects/Information_Measurement/Programs"
sys.path.insert(1, add_dir)

from class_signal import Signal
from Useful_Funtions_AS import load_object

# Define a function to remove .0 from integers in a latex matrix
def remove_dot_zero(latex_matrix):
  # Split the latex matrix by newline characters
  lines = latex_matrix.split("\n")
  # Initialize an empty list to store the modified lines
  new_lines = []
  # Loop through each line
  for line in lines:
    # Split the line by spaces
    elements = line.split()
    # Initialize an empty list to store the modified elements
    new_elements = []
    # Loop through each element
    for element in elements:
      # Try to convert the element to a float
      try:
        num = float(element)
        # If successful, check if it is an integer
        if num.is_integer():
          # If yes, convert it back to an integer and append it as a string
          new_elements.append(str(int(num)))
        else:
          # If no, append it as it is
          new_elements.append(element)
      except ValueError:
        # If not a valid float, append it as it is
        new_elements.append(element)
    # Join the modified elements by spaces and append them to the new lines list
    new_lines.append(" ".join(new_elements))
  # Join the new lines by newline characters and return them as a string  
  return "\n".join(new_lines)

# General Function

# Import data

In [6]:
report_path = "Data_and_Models/ELIO_Article_Files/Tables/results_report.csv"
df = pd.read_csv(report_path)
df.describe()

# Add name column
patients = df['Seizure'].apply(lambda text: int(text[3:5]))
df['patient'] = patients

IM_scores_column_indx = [indx for indx in df.columns if indx.find("_Score")!=-1]


In [13]:
# EEG detected results
for column_indx in IM_scores_column_indx:
    results = df[column_indx].apply(lambda x: x > 0.90)
    df['Results_%s'%(column_indx)] = results

In [14]:
seizure_detected_per_pacient = []
for column_indx in IM_scores_column_indx:
    seizure_detected_per_pacient.append(df.groupby("patient")['Results_%s'%(column_indx)].sum())

In [15]:
# Generate df with the number of seizure detected organized by patient
df_results = seizure_detected_per_pacient[0]
for ii in range(1, len(seizure_detected_per_pacient)):
    df_results = pd.concat([df_results, seizure_detected_per_pacient[ii]], axis=1)

df_results = pd.concat([df_results, df.groupby("patient")['Seizure'].size().rename('N_Seizures')], axis=1)

# Add a line with the total number of seizure detected X IMs
new_row = df_results.sum()
new_row.name = 'Detected'
# Append the new row to the DataFrame
#df_results = df_results.append(new_row, ignore_index=True)
df_results = df_results.append(new_row.astype(int))

# Add a line with the total number of seizure detected X IMs
new_row = 100*df_results.sum()/df_results.sum()['N_Seizures']
new_row.name = 'Detected $[\%]$'
# Append the new row to the DataFrame
#df_results = df_results.append(new_row, ignore_index=True)
df_results = df_results.append(new_row)

df_results = df_results.applymap(lambda x: int(x) if x == int(x) else x)

#pd.options.display.float_format = '{:.0f}'.format

# move the last column to the second position
cols = list(df_results.columns)
cols.insert(0, cols.pop())
df_results = df_results[cols]

df_results


/var/folders/7z/qxlqrhp92j30cpy4m2dbjn4r0000gn/T/ipykernel_92007/4004913455.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(new_row.astype(int))
/var/folders/7z/qxlqrhp92j30cpy4m2dbjn4r0000gn/T/ipykernel_92007/4004913455.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results = df_results.append(new_row)


,N_Seizures,Results_Shannon_Score,Results_Best_Tsallis_Score_q=0.7,Results_Best_Tsallis_Score_q=1.5,Results_Best_Tsallis_Score_q=2,Results_Best_Tsallis_Score_q=3,Results_Best_Tsallis_Score_q=4,Results_Best_Reyni_Score_q=0.7,Results_Best_Reyni_Score_q=1.5,Results_Best_Reyni_Score_q=2,Results_Best_Reyni_Score_q=3,Results_Best_Reyni_Score_q=4,Results_Best_Fisher_Score
patient,,,,,,,,,,,,,
1,7,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
2,3,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
3,7,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
4,4,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,4.000000,4.000000,4.000000,3.000000
5,5,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
6,10,10.000000,10.000000,10.000000,10.000000,9.000000,9.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
7,3,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
8,5,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
9,4,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [31]:
p_results = df_results.sum()/df_results.sum()['N_Seizures']
N_tot_seizure = df_results.sum()['N_Seizures']
p_0 = 0.5 #Change it with the time proportion

IM_ber_test = np.zeros(len(p_results))
for ii, p_result in enumerate(p_results):
    IM_ber_test[ii] = Signal.bernoulli_hypothesis_test(p_result, p_0, N_tot_seizure, direction='greater')

#df_results['N_Seizures'][ii+1]
#Signal.bernoulli_hypothesis_test(p_sample, p_0, df_results.sum()['N_Seizures'], direction='greater')

In [32]:
# Compute % Values
percentage_results = []
for column_indx in IM_scores_column_indx:
    #percentage_results.append( (df_results['Results_%s'%(column_indx)]/df_results['N_Seizures']).rename('Per_Results_%s'%(column_indx)).apply(lambda x: '{:.2}\%'.format(x)))
    percentage_results.append( (df_results['Results_%s'%(column_indx)]/df_results['N_Seizures']).rename('Per_Results_%s'%(column_indx)).apply(lambda x: x*100))
    #percentage_results.append((df_results['Results_%s'%(column_indx)]/df_results['N_Seizures']).rename('Per_Results_%s'%(column_indx)))

df_per_results = percentage_results[0]
for ii in range(1, len(percentage_results)):
    df_per_results = pd.concat([df_per_results, percentage_results[ii]], axis=1)
    
#df_per_results = pd.concat([df_per_results, df.groupby("patient")['Seizure'].size().rename('N_Seizures')], axis=1)


# LATEX TABLES

In [36]:
# Save  Latex Matrices
report_path = 'Data_and_Models/ELIO_Article_Files/Tables/N_seizure_detected_result.txt'

header = [["N_Seizures", r"$S$", r"$T$", r"$T$", r"$T$", r"$T$", r"$T$", r"$R$", r"$R$", r"$R$", r"$R$", r"$R$", r"$F$"],
            [" ", " ", r"$0.7$", r"$1.5$", r"$2$", r"$3$", r"$4$", r"$0.7$", r"$1.5$", r"$2$", r"$3$", r"$4$", " "]]

df_results.columns = header

with open(report_path, 'w') as file:
    file.write(
        df_results
        .reset_index()
        .round(1)
        #.rename(columns={"old_column_name" : "new_column_name"})
        #.reset_index()
        .to_latex(
            index=False,
            caption="Seizure detected",
            label="tab:per_Value",
            escape=False,
            column_format="lccccccccccccc",
            multicolumn_format="l"
        )
    )
    
# Read a latex matrix from a txt file (assuming it has bmatrix format)
with open(report_path, "r") as f:
  latex_matrix = f.read()

new_latex_matrix = remove_dot_zero(latex_matrix)

# Write the modified latex matrix to another txt file (keeping bmatrix format)
with open(report_path, "w") as f:
  f.write(new_latex_matrix)

/var/folders/7z/qxlqrhp92j30cpy4m2dbjn4r0000gn/T/ipykernel_87091/1676130031.py:11: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_results


In [37]:
print(new_latex_matrix)

\begin{table}
\centering
\caption{Seizure detected}
\label{tab:per_Value}
\begin{tabular}{lccccccccccccc}
\toprule
patient & N_Seizures & $S$ & \multicolumn{5}{c}{$T$} & \multicolumn{5}{c}{$R$} & $F$ \\
& $0.7$ & $1.5$ & $2$ & $3$ & $4$ & $0.7$ & $1.5$ & $2$ & $3$ & \multicolumn{3}{c}{$4$} \\
\midrule
1 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 \\
2 & 3 & 2 & 3 & 1 & 1 & 1 & 1 & 3 & 1 & 1 & 1 & 1 & 2 \\
3 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 \\
4 & 4 & 2 & 4 & 2 & 3 & 3 & 3 & 4 & 2 & 3 & 3 & 3 & 3 \\
5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 & 5 \\
6 & 10 & 10 & 10 & 9 & 9 & 9 & 9 & 10 & 9 & 9 & 9 & 9 & 9 \\
7 & 3 & 3 & 3 & 3 & 3 & 3 & 3 & 3 & 3 & 3 & 3 & 3 & 3 \\
8 & 5 & 4 & 4 & 5 & 5 & 5 & 5 & 4 & 4 & 5 & 5 & 5 & 5 \\
9 & 4 & 4 & 4 & 4 & 4 & 4 & 4 & 4 & 4 & 4 & 4 & 4 & 4 \\
10 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 & 7 \\
11 & 3 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 2 \\
12 & 40 & 28 & 29 & 29 & 31 & 32 & 32 & 29 & 29 & 30 & 31 & 32 &

In [38]:
# Save Per Latex Matrices
report_path = 'Data_and_Models/ELIO_Article_Files/Tables/N_seizure_per_detected_result.txt'

header = [[r"$S$ [\%]", r"$T$[\%]", r"$T$ [\%]", r"$T$ [\%]", r"$T$ [\%]", r"$T$ [\%]", r"$R$ [\%]", r"$R$ [\%]", r"$R$ [\%]", r"$R$ [\%]", r"$R$ [\%]", r"$F$ [\%]"],
            [" ", r"$0.7$", r"$1.5$", r"$2$", r"$3$", r"$4$", r"$0.7$", r"$1.5$", r"$2$", r"$3$", r"$4$", " "]]
df_per_results.columns = header

with open(report_path, 'w') as file:
    file.write(
        df_per_results
        .reset_index()
        .round(1)
        #.rename(columns={"old_column_name" : "new_column_name"})
        #.reset_index()
        .to_latex(
            index=False,
            caption="Seizure detected",
            label="tab:per_Value",
            escape=False,
            column_format="lcccccccccccc",
            multicolumn_format="c"
        )
    )
    
# Read a latex matrix from a txt file (assuming it has bmatrix format)
with open(report_path, "r") as f:
  latex_matrix = f.read()

new_latex_matrix = remove_dot_zero(latex_matrix)

# Write the modified latex matrix to another txt file (keeping bmatrix format)
with open(report_path, "w") as f:
  f.write(new_latex_matrix)

/var/folders/7z/qxlqrhp92j30cpy4m2dbjn4r0000gn/T/ipykernel_87091/3509884343.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_per_results
